### From Multi-GPU Toy Example

In [1]:
# Standard Libraries
import os
import glob
import shutil

# External Dependencies
import cupy as cp
import cudf
import dask_cudf
from dask_cuda import LocalCUDACluster
from dask.distributed import Client
from dask.utils import parse_bytes
from dask.delayed import delayed
import rmm

# NVTabular
import nvtabular as nvt
import nvtabular.ops as ops
from nvtabular.io import Shuffle
from nvtabular.utils import device_mem_size

In [2]:
# Choose a "fast" root directory for this example
BASE_DIR = os.environ.get("BASE_DIR", "./basedir")

# Define and clean our worker/output directories
dask_workdir = os.path.join(BASE_DIR, "workdir")
demo_output_path = os.path.join(BASE_DIR, "demo_output")
demo_dataset_path = os.path.join(BASE_DIR, "demo_dataset")

# Ensure BASE_DIR exists
if not os.path.isdir(BASE_DIR):
    os.mkdir(BASE_DIR)

# Make sure we have a clean worker space for Dask
if os.path.isdir(dask_workdir):
    shutil.rmtree(dask_workdir)
os.mkdir(dask_workdir)

# Make sure we have a clean output path
if os.path.isdir(demo_output_path):
    shutil.rmtree(demo_output_path)
os.mkdir(demo_output_path)

# Get device memory capacity
capacity = device_mem_size(kind="total")

In [3]:
# Deploy a Single-Machine Multi-GPU Cluster
protocol = "tcp"             # "tcp" or "ucx"
visible_devices = "0,1,2"  # Delect devices to place workers
device_spill_frac = 0.5      # Spill GPU-Worker memory to host at this limit.
                             # Reduce if spilling fails to prevent
                             # device memory errors.
cluster = None               # (Optional) Specify existing scheduler port
if cluster is None:
    cluster = LocalCUDACluster(
        protocol = protocol,
        CUDA_VISIBLE_DEVICES = visible_devices,
        local_directory = dask_workdir,
        device_memory_limit = capacity * device_spill_frac,
    )

# Create the distributed client
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:38039 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 3 Cores: 3 Memory: 135.10 GB


In [4]:
# Initialize RMM pool on ALL workers
def _rmm_pool():
    rmm.reinitialize(
        pool_allocator=True,
        initial_pool_size=None, # Use default size
    )
    
client.run(_rmm_pool)

{'tcp://127.0.0.1:33157': None,
 'tcp://127.0.0.1:37955': None,
 'tcp://127.0.0.1:38567': None}

### From Getting Started MovieLens

#### Getting Started

In [5]:
# External dependencies
import os
import cudf                 # cuDF is an implementation of Pandas-like Dataframe on GPU
import time
import gc

from os import path
from sklearn.model_selection import train_test_split

In [6]:
BASE_DIR = './data'

In [7]:
if not path.exists(BASE_DIR + 'ml-25m'):
    os.makedirs(BASE_DIR, exist_ok=True)
    zip_path = os.path.join(BASE_DIR, 'ml-25m.zip')
    if not path.exists(zip_path):
        os.system("mkdir -p " + BASE_DIR)
        os.system("wget http://files.grouplens.org/datasets/movielens/ml-25m.zip")
        os.system("mv ml-25m.zip " + BASE_DIR)
    
    os.system("unzip " + zip_path + " -d " + BASE_DIR)

#### Convert the dataset

In [8]:
movies = cudf.read_csv(os.path.join(BASE_DIR, 'ml-25m/movies.csv'))

In [9]:
movies['genres'] = movies['genres'].str.split('|')
movies = movies.drop('title', axis=1)

In [10]:
movies.to_parquet(os.path.join(BASE_DIR, "ml-25m", "movies_converted.parquet"))

#### Splitting into train and validation dataset

In [11]:
ratings = cudf.read_csv(os.path.join(BASE_DIR, "ml-25m", "ratings.csv"))

In [12]:
ratings = ratings.drop('timestamp', axis=1)
train, valid = train_test_split(ratings, test_size=0.2, random_state=42)

In [13]:
joined = (
    ['userId', 'movieId'] >> 
    nvt.ops.JoinExternal(movies, on=['movieId'])
)

In [14]:
train.to_parquet(os.path.join(BASE_DIR, "train.parquet"))
valid.to_parquet(os.path.join(BASE_DIR, "valid.parquet"))

### ETL with NVTabular

In [15]:
movies= cudf.read_parquet(os.path.join(BASE_DIR, "ml-25m", "movies_converted.parquet"))

#### Defining our Preprocessing Pipeline

In [16]:
cat_features = joined >> nvt.ops.Categorify()

In [17]:
ratings = nvt.ColumnGroup(['rating']) >> (lambda col: (col>3).astype('int8'))

In [18]:
output = cat_features+ratings

In [19]:
# THIS CELL NOW USES THE DASK CLUSTER CLIENT CREATED ABOVE
workflow = nvt.Workflow(output, client=client)

#### Running the pipeline

In [20]:
!rm -r $BASE_DIR/train
!rm -r $BASE_DIR/valid

In [21]:
train_iter = nvt.Dataset([os.path.join(BASE_DIR, "train.parquet")], part_size="100MB")
valid_iter = nvt.Dataset([os.path.join(BASE_DIR, "valid.parquet")], part_size="100MB")

In [22]:
workflow.fit(train_iter)
workflow.save(os.path.join(BASE_DIR, "workflow"))

/opt/conda/envs/rapids/lib/python3.8/site-packages/distributed/worker.py:3451: UserWarning: Large object of size 1.99 MB detected in task graph: 
  ("('read-parquet-b80b0101737ef3ff0711e012e3b2445e' ... 1e012e3b2445e')
Consider scattering large objects ahead of time
with client.scatter to reduce scheduler burden and 
keep data on workers

    future = client.submit(func, big_data)    # bad

    big_future = client.scatter(big_data)     # good
    future = client.submit(func, big_future)  # good
  warnings.warn(


In [23]:
%%time
shuffle = Shuffle.PER_WORKER  # Shuffle algorithm
out_files_per_proc = 4        # Number of output files per worker
workflow.transform(train_iter).to_parquet(
    output_path=os.path.join(BASE_DIR, "train"),
    shuffle=shuffle,
    out_files_per_proc=out_files_per_proc,
)

CPU times: user 182 ms, sys: 27.9 ms, total: 210 ms
Wall time: 2.03 s


In [24]:
%%time
shuffle = Shuffle.PER_WORKER  # Shuffle algorithm
out_files_per_proc = 4        # Number of output files per worker
workflow.transform(valid_iter).to_parquet(
    output_path=os.path.join(BASE_DIR, "valid"),
    shuffle=shuffle,
    out_files_per_proc=out_files_per_proc,
)

CPU times: user 106 ms, sys: 1.02 ms, total: 107 ms
Wall time: 711 ms


#### Checking the pre-processing outputs

In [25]:
import glob

TRAIN_PATHS = sorted(glob.glob(os.path.join(BASE_DIR, "train", "*.parquet")))
VALID_PATHS = sorted(glob.glob(os.path.join(BASE_DIR, "valid", "*.parquet")))
TRAIN_PATHS, VALID_PATHS

(['./data/train/0.2d58cf8e6f434baeb73e20752c94bd89.parquet',
  './data/train/0.3b7a7e47f6d74a08a5de89a0afea9a7b.parquet',
  './data/train/0.57b1368959eb4f4aa2ebc4bdca44c9f5.parquet',
  './data/train/1.7f3b8d62b89d4febb2111ac02c288b85.parquet',
  './data/train/1.c1b2cb626efa469db9050ffd6ab0b0e3.parquet',
  './data/train/1.eef1ad42f18c4831840b6d941ed8f5d4.parquet',
  './data/train/2.1b403fb26dbd43569caecdf4cb8c1ce8.parquet',
  './data/train/2.28615e85225e42108965df9632839f9a.parquet',
  './data/train/2.dd843a4d1aa44c0791f0321962401f06.parquet',
  './data/train/3.56a83f9bf947441a99d12374c2a0087f.parquet',
  './data/train/3.78e4e1c853c645f798c94062040d1e55.parquet',
  './data/train/3.e7990c31c17e4cef967988f40bacb254.parquet'],
 ['./data/valid/0.40eb916137194b0583c69b653c87cb67.parquet',
  './data/valid/0.630a271d81cc4c15980a96029c48714d.parquet',
  './data/valid/1.174b8b6418ec44fcb9283cf44b4add68.parquet',
  './data/valid/1.c844640f238947dbaf747a9ad3c32e2a.parquet',
  './data/valid/2.5610b

In [26]:
df = cudf.read_parquet(TRAIN_PATHS[0])
df.head()

,userId,movieId,genres,rating
0,105207,5844,"[7, 9]",0
1,51304,2677,"[7, 9]",0
2,13068,3923,"[3, 6, 7]",1
3,43489,14022,"[3, 4, 5, 6, 17]",0
4,73912,1172,"[2, 3, 20]",1


### From Tensorflow Multi-GPU Training Example

In [31]:
%%writefile './hvd_wrapper.sh'

#!/bin/bash

## Get local process ID from OpenMPI or alternatively from SLURM
if [ -z "${CUDA_VISIBLE_DEVICES:-}" ]; then
    if [ -n "${OMPI_COMM_WORLD_LOCAL_RANK:-}" ]; then
        LOCAL_RANK="${OMPI_COMM_WORLD_LOCAL_RANK}"
    elif [ -n "${SLURM_LOCALID:-}" ]; then
        LOCAL_RANK="${SLURM_LOCALID}"
    fi
    export CUDA_VISIBLE_DEVICES=${LOCAL_RANK}
fi

exec "$@"

Overwriting ./hvd_wrapper.sh


In [32]:
%%writefile './trainer.py'

# External dependencies
import argparse
import glob
import os

import cupy

# we can control how much memory to give tensorflow with this environment variable
# IMPORTANT: make sure you do this before you initialize TF's runtime, otherwise
# TF will have claimed all free GPU memory
os.environ["TF_MEMORY_ALLOCATION"] = "0.3"  # fraction of free memory
import horovod.tensorflow as hvd  # noqa: E402
import tensorflow as tf  # noqa: E402

import nvtabular as nvt  # noqa: E402
from nvtabular.framework_utils.tensorflow import layers  # noqa: E402
from nvtabular.loader.tensorflow import KerasSequenceLoader  # noqa: E402

parser = argparse.ArgumentParser(description="Process some integers.")
parser.add_argument("--dir_in", default=None, help="Input directory")
parser.add_argument("--b_size", default=None, help="batch size")
parser.add_argument("--cats", default=None, help="categorical columns")
parser.add_argument("--cats_mh", default=None, help="categorical multihot columns")
parser.add_argument("--conts", default=None, help="continuous columns")
parser.add_argument("--labels", default=None, help="continuous columns")
args = parser.parse_args()


BASE_DIR = args.dir_in or "./data/"
BATCH_SIZE = args.b_size or 16384  # Batch Size
CATEGORICAL_COLUMNS = args.cats or ["movieId", "userId"]  # Single-hot
CATEGORICAL_MH_COLUMNS = args.cats_mh or ["genres"]  # Multi-hot
NUMERIC_COLUMNS = args.conts or []
TRAIN_PATHS = sorted(
    glob.glob(os.path.join(BASE_DIR, "train/*.parquet"))
)  # Output from ETL-with-NVTabular
hvd.init()

# Seed with system randomness (or a static seed)
cupy.random.seed(None)


def seed_fn():
    """
    Generate consistent dataloader shuffle seeds across workers

    Reseeds each worker's dataloader each epoch to get fresh a shuffle
    that's consistent across workers.
    """
    min_int, max_int = tf.int32.limits
    max_rand = max_int // hvd.size()

    # Generate a seed fragment
    seed_fragment = cupy.random.randint(0, max_rand).get()

    # Aggregate seed fragments from all Horovod workers
    seed_tensor = tf.constant(seed_fragment)
    reduced_seed = hvd.allreduce(seed_tensor, name="shuffle_seed", op=hvd.mpi_ops.Sum) % max_rand

    return reduced_seed


proc = nvt.Workflow.load(os.path.join(BASE_DIR, "workflow/"))
EMBEDDING_TABLE_SHAPES = nvt.ops.get_embedding_sizes(proc)

train_dataset_tf = KerasSequenceLoader(
    TRAIN_PATHS,  # you could also use a glob pattern
    batch_size=BATCH_SIZE,
    label_names=["rating"],
    cat_names=CATEGORICAL_COLUMNS + CATEGORICAL_MH_COLUMNS,
    cont_names=NUMERIC_COLUMNS,
    engine="parquet",
    shuffle=True,
    seed_fn=seed_fn,
    buffer_size=0.06,  # how many batches to load at once
    parts_per_chunk=1,
    global_size=hvd.size(),
    global_rank=hvd.rank(),
)
inputs = {}  # tf.keras.Input placeholders for each feature to be used
emb_layers = []  # output of all embedding layers, which will be concatenated
for col in CATEGORICAL_COLUMNS:
    inputs[col] = tf.keras.Input(name=col, dtype=tf.int32, shape=(1,))
# Note that we need two input tensors for multi-hot categorical features
for col in CATEGORICAL_MH_COLUMNS:
    inputs[col + "__values"] = tf.keras.Input(name=f"{col}__values", dtype=tf.int64, shape=(1,))
    inputs[col + "__nnzs"] = tf.keras.Input(name=f"{col}__nnzs", dtype=tf.int64, shape=(1,))
for col in CATEGORICAL_COLUMNS + CATEGORICAL_MH_COLUMNS:
    emb_layers.append(
        tf.feature_column.embedding_column(
            tf.feature_column.categorical_column_with_identity(
                col, EMBEDDING_TABLE_SHAPES[col][0]  # Input dimension (vocab size)
            ),
            EMBEDDING_TABLE_SHAPES[col][1],  # Embedding output dimension
        )
    )
emb_layer = layers.DenseFeatures(emb_layers)
x_emb_output = emb_layer(inputs)
x = tf.keras.layers.Dense(128, activation="relu")(x_emb_output)
x = tf.keras.layers.Dense(128, activation="relu")(x)
x = tf.keras.layers.Dense(128, activation="relu")(x)
x = tf.keras.layers.Dense(1, activation="sigmoid")(x)
model = tf.keras.Model(inputs=inputs, outputs=x)
loss = tf.losses.BinaryCrossentropy()
opt = tf.keras.optimizers.SGD(0.01 * hvd.size())
opt = hvd.DistributedOptimizer(opt)
checkpoint_dir = "./checkpoints"
checkpoint = tf.train.Checkpoint(model=model, optimizer=opt)


@tf.function(experimental_relax_shapes=True)
def training_step(examples, labels, first_batch):
    with tf.GradientTape() as tape:
        probs = model(examples, training=True)
        loss_value = loss(labels, probs)
    # Horovod: add Horovod Distributed GradientTape.
    tape = hvd.DistributedGradientTape(tape, sparse_as_dense=True)
    grads = tape.gradient(loss_value, model.trainable_variables)
    opt.apply_gradients(zip(grads, model.trainable_variables))
    # Horovod: broadcast initial variable states from rank 0 to all other processes.
    # This is necessary to ensure consistent initialization of all workers when
    # training is started with random weights or restored from a checkpoint.
    #
    # Note: broadcast should be done after the first gradient step to ensure optimizer
    # initialization.
    if first_batch:
        hvd.broadcast_variables(model.variables, root_rank=0)
        hvd.broadcast_variables(opt.variables(), root_rank=0)
    return loss_value


# Horovod: adjust number of steps based on number of GPUs.
for batch, (examples, labels) in enumerate(train_dataset_tf):
    loss_value = training_step(examples, labels, batch == 0)
    if batch % 10 == 0 and hvd.local_rank() == 0:
        print("Step #%d\tLoss: %.6f" % (batch, loss_value))
hvd.join()
# Horovod: save checkpoints only on worker 0 to prevent other workers from
# corrupting it.
if hvd.rank() == 0:
    checkpoint.save(checkpoint_dir)


Overwriting ./trainer.py


In [33]:
!horovodrun -np 3 sh hvd_wrapper.sh python trainer.py --dir_in $BASE_DIR --b_size 1024

2021-04-01 20:21:54.254145: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
[1,1]<stderr>:2021-04-01 20:21:56.271981: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
[1,0]<stderr>:2021-04-01 20:21:56.291559: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
[1,2]<stderr>:2021-04-01 20:21:56.291559: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
[1,1]<stderr>:2021-04-01 20:21:58.752873: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
[1,1]<stderr>:2021-04-01 20:21:58.753069: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
[1,1]<stderr>:2021-04-01 20:21:58.755666: I tensorflow/core/common_ru